In [1]:
import redis
import pickle 
import torch
database = redis.Redis(host='localhost', port=6379)

In [2]:
import glob
from tqdm import tqdm
import torch
feature_dirs = glob.glob('/remote-home/share/DATA/tcga-4096-processed/features/pt_files/*.pt')
features = [] 
for x in tqdm(feature_dirs):
    features.append(torch.load(x))

100%|██████████| 1041/1041 [02:10<00:00,  7.95it/s]


### load data into redis for pretraining [384, 16, 16]

In [7]:
from tqdm import tqdm
import pickle
import json
feature_ind_map = {}
dataset_ind = 0
for wsi_ind, (feature_dir, feature) in tqdm(enumerate(zip(feature_dirs, features))):
    slide_id = feature_dir.split('/')[-1][:-3] # e.g. 'TCGA-85-7698-01Z-00-DX1.bc8a89a7-356e-4913-9632-627774730f48'
    region_num_this_wsi = len(feature)
    
    for region_ind in range(region_num_this_wsi):
        region_feature = feature[region_ind] # Tensor, [384, 16, 16]
        feature_ind_map[dataset_ind] = {'slide_id': slide_id, 'slide_index': wsi_ind, 'region_index': region_ind, 'dataset_index': dataset_ind}
        region_feature_bytes = pickle.dumps(region_feature.numpy())
        database.set(dataset_ind, region_feature_bytes)
        dataset_ind += 1
print(dataset_ind)
with open('tcga_lung_4k.json','w', encoding='utf8') as f:
    json.dump(feature_ind_map, f,   ensure_ascii=False,  indent=2)

1041it [00:57, 17.96it/s]


62853


### load data into redis for downsteam tasks [N, 384, 16, 16]

In [3]:
from tqdm import tqdm
import pickle
import json
import redis
import pickle 
import torch
database = redis.Redis(host='localhost', port=6379)
feature_ind_map = {}
dataset_ind = 100000 # 62852
for wsi_ind, (feature_dir, feature) in tqdm(enumerate(zip(feature_dirs, features))):
    slide_id = feature_dir.split('/')[-1][:-3] # e.g. 'TCGA-85-7698-01Z-00-DX1.bc8a89a7-356e-4913-9632-627774730f48'
    feature_ind_map[dataset_ind] = {'slide_id': slide_id, 'slide_index': wsi_ind, 'dataset_index': dataset_ind}
    feature_bytes = pickle.dumps(feature.numpy())
    database.set(dataset_ind, feature_bytes)
    dataset_ind += 1
    # del feature
print(dataset_ind)
with open('tcga_lung_WSI.json','w', encoding='utf8') as f:
    json.dump(feature_ind_map, f,   ensure_ascii=False,  indent=2)

1041it [01:38, 10.57it/s]

101041
